In [1]:
import os, random
import numpy as np
import cv2
import json 
import pandas as pd


In [6]:
import string
def convert_to_lable(data):
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    alphabet28 = string.ascii_lowercase + ' _' # 26 is space, 27 is CTC blank char
    alphabet87 = string.ascii_lowercase + string.ascii_uppercase + string.digits + ' +-*.,:!?%&$~/()[]<>"\'@#_'
    return list(map(lambda x: alphabet87.index(x), data))
convert_to_lable('afsfs[]')
len(string.ascii_lowercase + string.ascii_uppercase + string.digits + ' +-*.,:!?%&$~/()[]<>"\'@#_')





87

In [7]:
class crnnGenerator:
    def __init__(self,imgpath,labelpath,imgw,imgh,batch_size,maxlen,inputlen):
        self.imgw=imgw
        self.imgh=imgh
        self.batch_size=batch_size
        self.imgpath=imgpath
        self.imgdir=os.listdir(self.imgpath)
        self.imgcount=len(self.imgdir)
        self.index=list(range(self.imgcount))
        self.curindex=0
        self.maxlen=maxlen
        self.imgs=np.zeros((self.imgcount,self.imgh,self.imgw))
        self.texts=[]
        self.labelpath=labelpath
        self.labelpd=pd.read_json(self.labelpath,orient='index')
        self.inputlen=inputlen
    
    def build_data(self):
        for i,file in enumerate(self.imgdir):
            a=self.labelpd.loc[self.labelpd['cropimgid']==file[0:-4] ]['label']
            if (len(a.values[0])) <self.maxlen:
                img=cv2.imread(self.imgpath+file,cv2.IMREAD_GRAYSCALE)
                img=cv2.resize(img,(self.imgw,self.imgh))
                img=img.astype(np.float32)
                img=(img / 255.0)

                self.imgs[i,:,:]=img

                


                self.texts.append(convert_to_lable(a.values[0]))
            else:
                self.imgcount=self.imgcount-1
        self.index=list(range(self.imgcount)) 
    def next_sample(self):
        self.curindex+=1
        if self.curindex>=self.imgcount:
            self.curindex=0
            random.shuffle(self.index)
        
        a=self.imgs[self.index[self.curindex]]
        b=self.texts[self.index[self.curindex]]
        return a,b
    
    def test(self,i=1,j=2):
        a=self.imgs[self.index[self.curindex]]
        b=self.texts[self.index[self.curindex]]
        return a,b
    
    def texttest(self,file='000adfe5b817011c_1'):
        img=cv2.imread(self.imgpath+file+'.jpg',cv2.IMREAD_GRAYSCALE)

        img=cv2.resize(img,(self.imgw,self.imgh))
        img=img.astype(np.float32)
        img=img/255.0
        cv2.imshow('ada',img)
        cv2.waitKey()
        cv2.destroyAllWindows()
    
    def next_batch(self):
        while True:
            xdata=np.ones([self.batch_size,self.imgw,self.imgh,1])
            ydata=np.ones([self.batch_size,self.maxlen])
            inputlength=np.zeros((self.batch_size,1))   #(self.imgw//2) 최대길이
            labellength=np.zeros((self.batch_size,1))
            
            for i in range(self.batch_size):
                img,text=self.next_sample()
                
                img=img.T
                img=np.expand_dims(img,-1)
                xdata[i]=img
                for j,k in enumerate(text):
                    
                    
                    ydata[i][j]=k
                inputlength[i]=self.maxlen
                labellength[i]=len(text)
                
            inputs={
                'image_input':xdata,
                'labels':ydata,
                'input_length':inputlength,
                'label_length':labellength
            }
            outputs={'ctcloss':np.zeros([self.batch_size])}
            yield(inputs,outputs)
            
            
        
        

In [5]:
"""
cropvalpath='D:/engocr/abcd/'
croptrainpath='D:/engocr/abcd/'
cropvaljson='D:/engocr/valcrnn.json'
batch=32
traingen=crnnGenerator(imgpath=croptrainpath,labelpath=cropvaljson,imgw=128,imgh=64,batch_size=batch,
                      maxlen=100,inputlen=100)
print('trainstart')
traingen.build_data()
print('valstart')
valgen=crnnGenerator(imgpath=cropvalpath,labelpath=cropvaljson,imgw=128,imgh=64,batch_size=batch,
                      maxlen=100,inputlen=100)
valgen.build_data()
print('valend')

"""

trainstart
valstart
valend


In [6]:
#valgen.test()
#valgen.texttest()

(array([[0.60392159, 0.6156863 , 0.63529414, ..., 0.31764707, 0.27450982,
         0.24705882],
        [0.57254905, 0.54901963, 0.50980395, ..., 0.30588236, 0.26274511,
         0.23529412],
        [0.43137255, 0.40392157, 0.35686275, ..., 0.3137255 , 0.27058825,
         0.23921569],
        ...,
        [0.43137255, 0.44705883, 0.47058824, ..., 0.46666667, 0.46666667,
         0.46666667],
        [0.4509804 , 0.4509804 , 0.4509804 , ..., 0.46666667, 0.46666667,
         0.46666667],
        [0.46666667, 0.45882353, 0.44313726, ..., 0.46666667, 0.46666667,
         0.46666667]]),
 [28, 27, 28, 28, 26, 32, 27, 35, 34, 31])

In [7]:
#ada=cv2.resize(img,(self.imgw,self.imgh))

64

In [36]:
"""
y = np.ones([16, 100])
b=convert_to_lable('afsfs')
for i,j in enumerate(b):
    print(i,j)
    y[1][i]=j
y
"""

0 0
1 5
2 18
3 5
4 18


array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 0.,  5., 18., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

array([[9999., 9999., 9999., ..., 9999., 9999., 9999.],
       [9999., 9999., 9999., ..., 9999., 9999., 9999.],
       [9999., 9999., 9999., ..., 9999., 9999., 9999.],
       ...,
       [9999., 9999., 9999., ..., 9999., 9999., 9999.],
       [9999., 9999., 9999., ..., 9999., 9999., 9999.],
       [9999., 9999., 9999., ..., 9999., 9999., 9999.]])